In [1]:
from google.colab import drive
import sys
drive.mount('/content/drive', force_remount=True)
sys.path.append('/content/drive/MyDrive/attention-model-pl')

Mounted at /content/drive


In [2]:
%ls

drive/  sample_data/


In [3]:
%cd /content/drive/MyDrive/attention-model-pl

/content/drive/MyDrive/attention-model-pl


In [4]:
!pip install -r requirements.txt

In [5]:
%load_ext autoreload
%autoreload 2

In [6]:
%ls

config.py  lightning_logs/     models/       README.md         tokenizer_en.json  weights/
data/      lightning-train.py  notebooks/    requirements.txt  tokenizer_it.json
LICENSE    logs/               __pycache__/  runs/             utils/


In [7]:
from config import get_config

cfg = get_config()
cfg['batch_size'] = 6
cfg['preload'] = True


from data.data_module import *
from models.lightning_model import *
import pytorch_lightning as pl
from pytorch_lightning.callbacks import ModelCheckpoint, LearningRateMonitor
import torch

In [8]:
datamodule = BilingualDataModule(cfg)
datamodule.setup()

Max length of source language: 309
Max length of target language: 274


In [9]:
tk_src = datamodule.tokenizer_src  # Assuming datamodule is your instance of BilingualDataModule
tk_tgt = datamodule.tokenizer_tgt

src_vocab_size = tk_src.get_vocab_size()
tgt_vocab_size = tk_tgt.get_vocab_size()

model = LitModel(src_vocab_size, tgt_vocab_size)

In [10]:
from pytorch_lightning.loggers import TensorBoardLogger
from pytorch_lightning.callbacks.progress import TQDMProgressBar

In [11]:
# Tensorboard logger
tb_logger = TensorBoardLogger(save_dir="logs/", name="model")
# Initialize the Lightning Trainer

trainer = pl.Trainer(precision=16,max_epochs=cfg["num_epochs"],logger=tb_logger,
                     accelerator="auto",
        devices=1 if torch.cuda.is_available() else None,  # limiting got iPython runs
        callbacks=[
            ModelCheckpoint(dirpath=cfg['model_folder'], verbose=True),
            LearningRateMonitor(logging_interval="step", log_momentum=True)])

/usr/local/lib/python3.10/dist-packages/lightning_fabric/connector.py:565: `precision=16` is supported for historical reasons but its usage is discouraged. Please set your precision to 16-mixed instead!
INFO:pytorch_lightning.utilities.rank_zero:Using 16bit Automatic Mixed Precision (AMP)
INFO:pytorch_lightning.utilities.rank_zero:GPU available: True (cuda), used: True
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:IPU available: False, using: 0 IPUs
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs


In [ ]:
trainer.fit(model, datamodule)

Max length of source language: 309
Max length of target language: 274


/usr/local/lib/python3.10/dist-packages/pytorch_lightning/callbacks/model_checkpoint.py:630: Checkpoint directory /content/drive/MyDrive/attention-model-pl/weights exists and is not empty.
INFO:pytorch_lightning.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
INFO:pytorch_lightning.callbacks.model_summary:
  | Name  | Type        | Params
--------------------------------------
0 | model | Transformer | 75.1 M
--------------------------------------
75.1 M    Trainable params
0         Non-trainable params
75.1 M    Total params
300.532   Total estimated model params size (MB)


Sanity Checking: |          | 0/? [00:00<?, ?it/s]

/usr/local/lib/python3.10/dist-packages/pytorch_lightning/trainer/connectors/data_connector.py:492: Your `val_dataloader`'s sampler has shuffling enabled, it is strongly recommended that you turn shuffling off for val/test dataloaders.
/usr/local/lib/python3.10/dist-packages/pytorch_lightning/trainer/connectors/data_connector.py:441: The 'val_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=1` in the `DataLoader` to improve performance.


--------------------------------------------------------------------------------
SOURCE: 'Then how can you doubt that God created everything?' said the priest in puzzled amazement.
TARGET: Ma come potete dubitare che Dio abbia creato tutto? — disse il sacerdote con sorpresa ilare.
PREDICTED: avvolgeva avvolgeva avvolgeva accompagna accompagna accompagna accompagna accompagna accompagna accompagna accompagna accompagna accompagna accompagna accompagna accompagna accompagna accompagna accompagna accompagna accompagna accompagna accompagna accompagna accompagna accompagna accompagna accompagna accompagna accompagna accompagna accompagna accompagna accompagna accompagna accompagna accompagna accompagna accompagna accompagna accompagna accompagna accompagna accompagna accompagna accompagna accompagna accompagna accompagna accompagna accompagna accompagna accompagna accompagna accompagna accompagna accompagna accompagna accompagna accompagna accompagna raffreddasse fissandomi fissandomi avvo

...

--------------------------------------------------------------------------------
SOURCE: On the other hand, the man who wound it up thinks the whole cause of the muddle rests with the man who is trying to unwind it.
TARGET: D’altra parte, quello che l’ha arrotolato pensa che la cagione del danno sia tutta di chi ha tentato di svolgerlo.
PREDICTED: Il suo altro , che la mano si , è sempre più facile con la forza di , che è .
--------------------------------------------------------------------------------
SOURCE: After dinner, we immediately adjourned to the schoolroom: lessons recommenced, and were continued till five o'clock.
TARGET: Dopo desinare passammo subito nella sala di studio; le lezioni ricominciarono per durare fino alle cinque.
PREDICTED: Dopo pranzo , ci , ci la casa , e ci , e ci due .
--------------------------------------------------------------------------------
SOURCE: I was vexed to hear Harris go on in this blood-thirsty strain.
TARGET: Mi dispiaceva di sentire Harri

Validation: |          | 0/? [00:00<?, ?it/s]

--------------------------------------------------------------------------------
SOURCE: But it occurred to my thoughts, what call, what occasion, much less what necessity I was in to go and dip my hands in blood, to attack people who had neither done or intended me any wrong? who, as to me, were innocent, and whose barbarous customs were their own disaster, being in them a token, indeed, of God’s having left them, with the other nations of that part of the world, to such stupidity, and to such inhuman courses, but did not call me to take upon me to be a judge of their actions, much less an executioner of His justice—that whenever He thought fit He would take the cause into His own hands, and by national vengeance punish them as a people for national crimes, but that, in the meantime, it was none of my business—that it was true Friday might justify it, because he was a declared enemy and in a state of war with those very particular people, and it was lawful for him to attack them—but I

Exception in thread Thread-12:
Traceback (most recent call last):
  File "/usr/lib/python3.10/threading.py", line 1016, in _bootstrap_inner
    self.run()
  File "/usr/local/lib/python3.10/dist-packages/tensorboard/summary/writer/event_file_writer.py", line 244, in run
    self._run()
  File "/usr/local/lib/python3.10/dist-packages/tensorboard/summary/writer/event_file_writer.py", line 289, in _run
    self._record_writer.flush()
  File "/usr/local/lib/python3.10/dist-packages/tensorboard/summary/writer/record_writer.py", line 43, in flush
    self._writer.flush()
  File "/usr/local/lib/python3.10/dist-packages/tensorflow/python/lib/io/file_io.py", line 221, in flush
    self._writable_file.flush()
tensorflow.python.framework.errors_impl.FailedPreconditionError: logs/model/version_9/events.out.tfevents.1698327094.f32440f1ac50.5949.1; Transport endpoint is not connected
ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.

ERROR:root:I

Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/pytorch_lightning/trainer/call.py", line 44, in _call_and_handle_interrupt
    return trainer_fn(*args, **kwargs)
  File "/usr/local/lib/python3.10/dist-packages/pytorch_lightning/trainer/trainer.py", line 581, in _fit_impl
    self._run(model, ckpt_path=ckpt_path)
  File "/usr/local/lib/python3.10/dist-packages/pytorch_lightning/trainer/trainer.py", line 990, in _run
    results = self._run_stage()
  File "/usr/local/lib/python3.10/dist-packages/pytorch_lightning/trainer/trainer.py", line 1036, in _run_stage
    self.fit_loop.run()
  File "/usr/local/lib/python3.10/dist-packages/pytorch_lightning/loops/fit_loop.py", line 202, in run
    self.advance()
  File "/usr/local/lib/python3.10/dist-packages/pytorch_lightning/loops/fit_loop.py", line 359, in advance
    self.epoch_loop.run(self._data_fetcher)
  File "/usr/local/lib/python3.10/dist-packages/pytorch_lightning/loops/training_epoch_loop.py", line 137